In [80]:
import pandas as pd  # panda data handling
from datetime import datetime  # for dates

## Merging the Data 
hospitals = pd.read_csv("data/CaliforniaHospitalData.csv")  # hospital data from bryan
personnel = pd.read_csv("data/CaliforniaHospitalData_Personnel.txt", sep='\t')  # personnel data from bryan

merged = pd.merge(hospitals, personnel, on="HospitalID")  # join on HospitalID
merged.drop(columns=["Work_ID", "PositionID", "Website"], inplace=True)  # clean columns
print("Merged Data Preview:", merged.head())  # show merged data

Merged Data Preview:    HospitalID                              Name         Zip TypeControl  \
0       45740                  Mammoth Hospital  93546-0660    District   
1       12145  Victor Valley Community Hospital       92392  Non Profit   
2       25667        Pioneers Memorial Hospital       92227    District   
3       46996      Ridgecrest Regional Hospital       93555  Non Profit   
4       37393        Barstow Community Hospital       92311    Investor   

      Teaching DonorType  NoFTE    NetPatRev    InOperExp   OutOperExp  ...  \
0  Small/Rural   Charity  327.0  135520.2186  20523425.53  34916220.47  ...   
1  Small/Rural   Charity  345.0  136156.6913  33447542.78  20348596.22  ...   
2  Small/Rural   Charity  601.2  197094.2541  37254178.67  37832448.33  ...   
3  Small/Rural   Charity  400.0  139170.3798  23385570.10  24661355.90  ...   
4  Small/Rural   Charity  262.0  116797.8306  13684502.49  15159986.51  ...   

   AvlBeds   LastName  FirstName Gender             P

In [82]:
## Exporting the Data
# hospitals - Small/Rural, AvlBeds >= 15, OperInc >= 0
filtered = merged[(merged["TypeControl"] == "Small/Rural") & 
                   (merged["AvlBeds"] >= 15) & 
                   (merged["OperInc"] >= 0)]  # apply filters
filtered.to_csv("data/hospital_data_new.txt", sep='\t', index=False)  # export clean data
hospital_data = pd.read_csv("data/hospital_data_new.txt", sep='\t')  # import cleaned data
print("Filtered Export Preview:", filtered.head())  # show filtered data

Filtered Export Preview: Empty DataFrame
Columns: [HospitalID, Name, Zip, TypeControl, Teaching, DonorType, NoFTE, NetPatRev, InOperExp, OutOperExp, OperRev, OperInc, AvlBeds, LastName, FirstName, Gender, PositionTitle, Compensation, MaxTerm, StartDate, Phone, Email]
Index: []

[0 rows x 22 columns]


In [84]:
## Renaming Columns
hospital_data.rename(columns={
    "NoFTE": "FullTimeCount",
    "NetPatRev": "NetPatientRevenue",
    "InOperExp": "InpatientOperExp",
    "OutOperExp": "OutpatientOperExp",
    "OperRev": "Operating_Revenue",
    "OperInc": "Operating_Income"
}, inplace=True)  # rename asked columns
print("Renamed Columns Preview:", hospital_data.head())  # show renamed data

Renamed Columns Preview: Empty DataFrame
Columns: [HospitalID, Name, Zip, TypeControl, Teaching, DonorType, FullTimeCount, NetPatientRevenue, InpatientOperExp, OutpatientOperExp, Operating_Revenue, Operating_Income, AvlBeds, LastName, FirstName, Gender, PositionTitle, Compensation, MaxTerm, StartDate, Phone, Email]
Index: []

[0 rows x 22 columns]


In [88]:
## Inserting Records (simple and safe)
new_rows = pd.DataFrame([
    {
        "HospitalID": 99999,  # example hospital ID
        "LastName": "Dorband",
        "FirstName": "Jessica",
        "Gender": "F",
        "PositionTitle": "Regional Representative",
        "Compensation": 46978,
        "MaxTerm": 4,
        "StartDate": datetime.today().strftime('%m/%d/%Y')
    },
    {
        "HospitalID": 88888,  # another example ID
        "LastName": "Dorband",
        "FirstName": "Jessica",
        "Gender": "F",
        "PositionTitle": "Safety Inspection Member",
        "Compensation": 23987,
        "MaxTerm": 2,
        "StartDate": datetime.today().strftime('%m/%d/%Y')
    }
])
print("Inserted Rows Preview:\n", new_rows)

Inserted Rows Preview:
    HospitalID LastName FirstName Gender             PositionTitle  \
0       99999  Dorband   Jessica      F   Regional Representative   
1       88888  Dorband   Jessica      F  Safety Inspection Member   

   Compensation  MaxTerm   StartDate  
0         46978        4  04/14/2025  
1         23987        2  04/14/2025  


In [57]:
new_merge = pd.concat([hospital_data, new_rows], ignore_index=True)  # add new records
filtered_nonprofit = new_merge[
    (new_merge["TypeControl"] == "Non-Profit") &
    (new_merge["FullTimeCount"] > 250) &
    (new_merge["NetPatientRevenue"] >= 109000)]  # apply nonprofit constraints
no_employee_cols = filtered_nonprofit.drop(columns=["LastName", "FirstName", "Gender", "PositionTitle", "Compensation", "MaxTerm", "StartDate"])
regional_reps = new_merge[
    (new_merge["PositionTitle"] == "Regional Representative") &
    (new_merge["Operating_Income"] > 100000)]  # filter reps with one constraint
print("Non-Profit Filtered Preview:", no_employee_cols.head())  # show filtered nonprofit
print("Regional Rep Filtered Preview:", regional_reps.head())  # show regional reps

Non-Profit Filtered Preview: Empty DataFrame
Columns: [HospitalID, Name, Zip, TypeControl, Teaching, DonorType, FullTimeCount, NetPatientRevenue, InpatientOperExp, OutpatientOperExp, Operating_Revenue, Operating_Income, AvlBeds, Phone, Email]
Index: []
Regional Rep Filtered Preview: Empty DataFrame
Columns: [HospitalID, Name, Zip, TypeControl, Teaching, DonorType, FullTimeCount, NetPatientRevenue, InpatientOperExp, OutpatientOperExp, Operating_Revenue, Operating_Income, AvlBeds, LastName, FirstName, Gender, PositionTitle, Compensation, MaxTerm, StartDate, Phone, Email]
Index: []

[0 rows x 22 columns]


In [59]:
new_merge["StartDate"] = pd.to_datetime(new_merge["StartDate"])  # changing to datetime
dtypes = new_merge.dtypes  # data types - look at types
preview_dates = new_merge["StartDate"].head()  # date check
print("\nFirst 5 dates:\n", preview_dates)  # date check
print("\nColumn data types:\n", dtypes)  # confirm added conversion
print("\nRegional Reps with income > $100,000:\n", regional_reps[["FirstName", "LastName", "Operating_Income"]])  # last check - is this working?


First 5 dates:
 Series([], Name: StartDate, dtype: datetime64[ns])

Column data types:
 HospitalID                   object
Name                         object
Zip                          object
TypeControl                  object
Teaching                     object
DonorType                    object
FullTimeCount                object
NetPatientRevenue            object
InpatientOperExp             object
OutpatientOperExp            object
Operating_Revenue            object
Operating_Income             object
AvlBeds                      object
LastName                     object
FirstName                    object
Gender                       object
PositionTitle                object
Compensation                 object
MaxTerm                      object
StartDate            datetime64[ns]
Phone                        object
Email                        object
dtype: object

Regional Reps with income > $100,000:
 Empty DataFrame
Columns: [FirstName, LastName, Operating_Income]


In [ ]:
W3Schools. (n.d.). Python If...Else. https://www.w3schools.com/python/python_conditions.asp
pandas development team. (2024). pandas.DataFrame.merge. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
Python Software Foundation. (n.d.). 5. Data Structures — Python 3 documentation. https://docs.python.org/3/tutorial/datastructures.html